Ce notebook a pour but de donner un point de vue à l'aide des méthodes formelles les limites théoriques du jeu du blackjack. \
Nous allons pour cela utiliser la librairie "stormpy" qui permet de faire des calculs sur les chaînes de Markov. \
Pour ce faire, nous allons créer un dtmc (discrete-time Markov chain) qui représentera notre jeu du blackjack et à l'aide de propriétés, pouvoir tester cette limite théorique.

Pour la réallisation de ce notebook, je me suis basé sur la documentation de stormpy (https://moves-rwth.github.io/stormpy/) mais également sur leur exemple de DTMC (https://moves-rwth.github.io/stormpy/doc/models/building_dtmcs.html)

## 1. Importation des librairies

In [1]:
import gymnasium as gym # environnement.
import pickle # sauvegarde de notre modèle.
import stormpy # librairie pour les chaînes de Markov.

/home/fmsd/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment GymV26Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/fmsd/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:521: UserWarning: WARN: Overriding environment GymV22Environment-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


## 2. Création de l'environnement

In [2]:
# Crée l'environnement blackjack.
# natural = True : Si le joueur fait 21 avec deux cartes (As + 10), il obtient une reward plus élevée.
# sab = True : natural est ignoré (reward de 1 même si 21 avec deux cartes) et si le joueur ainsi que le croupier font 21, il s'agira d'un match nul.
env = gym.make('Blackjack-v1', natural=False, sab=True)

## 3. Création du DTMC

In [6]:
nb_poss = 19*21*2 # Toutes les possibilités du jeu (19 pour le joueur, 21 pour le croupier, 2 si As utilisable ou non).
nb_actions = 2 # Soit on pioche, soit on reste.

def make_dtmc_blackjack(n_obs, n_act):
    builder = stormpy.SparseMatrixBuilder(rows=0, columns=0, entries=0, force_dimension=False, has_custom_row_grouping=True, row_groups=0)
    
    # Label pour nos états.
    state_labeling = stormpy.storage.StateLabeling(nb_poss)
    labels = {"init", "win", "loose"}
    for label in labels:
        state_labeling.add_label(label)
        
    # Label pour les actions.
    action_labeling = stormpy.storage.ChoiceLabeling(nb_poss * nb_actions)
    labels = {"stick_1", "stick_2", "stick_3", "stick_4", "stick_5", "stick_6", "stick_7", "stick_8", "stick_9", "stick_10", "stick_11", "hit_1", "hit_2", "hit_3", "hit_4", "hit_5", "hit_6", "hit_7", "hit_8", "hit_9", "hit_10", "hit_11", "finish", "back"}
    for label in labels:
        action_labeling.add_label(label)
    
    for pc in range(4, 23): # 19 possibilités au total.
        for cr in range(2, 23): # 21 possibilités au total.
            # Matrice de transition.
            builder.new_row_group(((pc-4)*10 + (cr-2)) * nb_actions) # Sans As utilisable.
            builder.new_row_group(((pc+14)*10 + (cr-2)) * nb_actions) # Avec As utilisable.
            
            # Ajout des labels sur les états.
            if (pc <= 21) and (cr <= 11):
                state_labeling.add_label_to_state("init", (pc-4)*10 + (cr-2))
                state_labeling.add_label_to_state("init", (pc+14)*10 + (cr-2))
            elif ((17 <= cr) and (cr <= pc) and (pc <= 21)) or ((pc <= 21) and (cr == 22)):
                state_labeling.add_label_to_state("win", (pc-4)*10 + (cr-2))
                state_labeling.add_label_to_state("win", (pc+14)*10 + (cr-2))
            elif ((17 <= cr) and (pc <= cr)) or (pc == 22):
                state_labeling.add_label_to_state("loose", (pc-4)*10 + (cr-2))
                state_labeling.add_label_to_state("loose", (pc+14)*10 + (cr-2))
                
            # Ajout des actions.
            for action in range(nb_action):
                # action = stick = 0.
                if cr < 17:
                    if cr <= 10:
                        for carte in range(2, 12):
                            if carte == 10:
                                builder.add_next_value((pc-4)*10 + (cr-2), (pc-4)*10 + (cr-2) + carte, 2/5)
                                action_labeling.add_label_to_choice("stick_" + str(carte), (pc-4)*10 + (cr-2) + carte)
                            elif carte == 11:
                                builder.add_next_value((pc-4)*10 + (cr-2), (pc+14)*10 + (cr-2) + carte, 1/10)
                                action_labeling.add_label_to_choice("stick_" + str(carte), (pc-4)*10 + (cr-2) + carte)
                            else:
                                builder.add_next_value((pc-4)*10 + (cr-2), (pc-4)*10 + (cr-2) + carte, 1/10)
                                action_labeling.add_label_to_choice("stick_" + str(carte), (pc-4)*10 + (cr-2) + carte)
                    elif cr <= 12:
                        for carte in range(1, 11):
                            if carte == 10:
                                builder.add_next_value((pc-4)*10 + (cr-2), (pc-4)*10 + (cr-2) + carte, 2/5)
                                action_labeling.add_label_to_choice("stick_" + str(carte), (pc-4)*10 + (cr-2) + carte)
                            else:
                                builder.add_next_value((pc-4)*10 + (cr-2), (pc-4)*10 + (cr-2) + carte, 1/10)
                                action_labeling.add_label_to_choice("stick_" + str(carte), (pc-4)*10 + (cr-2) + carte)
                    else:
                        for carte in range(1, 11):
                            if carte >= (22 - cr):
                                # 10-carte pour les nombres qui font que carte + cr >= 22 et +4 pour J, D, R qui ne sont pas dedans car aussi de valeur 10.
                                builder.add_next_value((pc-4)*10 + (cr-2), (pc-4)*10 + (cr-2) + carte, (11 - carte + 4)/10)
                                for i in range(carte, 11):
                                    action_labeling.add_label_to_choice("stick_" + str(i), (pc-4)*10 + (cr-2) + carte)
                                # Tous les cas sont gérés.
                                break
                            else:
                                builder.add_next_value((pc-4)*10 + (cr-2), (pc-4)*10 + (cr-2) + carte, 1/10)
                                action_labeling.add_label_to_choice("stick_" + str(carte), (pc-4)*10 + (cr-2) + carte)
                else:
                    builder.add_next_value((pc-4)*10 + (cr-2), (pc-4)*10 + (cr-2), 1)
                    action_labeling.add_label_to_choice("finish", (pc-4)*10 + (cr-2))
                                
                
                #action = hit = 1.
                if pc <= 10:
                    for carte in range(2, 12):
                        if carte == 10:
                            builder.add_next_value((pc-4)*10 + (cr-2), (pc-4+carte)*10 + (cr-2), 2/5)
                            action_labeling.add_label_to_choice("hit_" + str(carte), (pc-4+carte)*10 + (cr-2))
                        elif carte == 11:
                            builder.add_next_value((pc-4)*10 + (cr-2), (pc+14+carte)*10 + (cr-2), 1/10)
                            action_labeling.add_label_to_choice("hit_" + str(carte), (pc+14+carte)*10 + (cr-2))
                        else:
                            builder.add_next_value((pc-4)*10 + (cr-2), (pc-4+carte)*10 + (cr-2), 1/10)
                            action_labeling.add_label_to_choice("hit_" + str(carte), (pc-4+carte)*10 + (cr-2))
                elif pc <= 12:
                    pass
    
    # Mettre tous les composents ensemble.
    components = stormpy.SparseModelComponents(transition_matrix=transition_matrix, state_labeling=state_labeling, rate_transition=False)
    components.choice_labeling = action_labeling
    
    # Créer notre MDP.
    dtmc = stormpy.storage.SparseDtmc(components)
    
    return dtmc